In [ ]:
import torch
import whisper

# Func to free up XPU VRAM from allocator
def clearvram():
    torch.xpu.memory.empty_cache()

# Clear VRAM
clearvram()

# Load a pre-trained model (e.g., "base", "small", "medium", "large")
model = whisper.load_model("turbo", device="xpu") 

# Transcribe your audio file
result = model.transcribe("test.wav")

print(result["text"])

# Clear VRAM
clearvram()

 Dancing in the masquerade, idle truth in plain sight jaded, pop, roll, click, shot, who will I be today or not? But such a tide as moving seems asleep, too full for sound and foam, when that which drew from out the boundless deep turns again home, twilight and evening bell, and after that?
